# Model Training

In [ ]:


!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

In [ ]:
# Login to WandB to see the performance of the training

%pip install -q wandb
import wandb
!wandb login


In [ ]:

# Train the model using the pretrained model yolov5s and the class labeled in securitycamera.yaml for 100 epochs
!python train.py --img 640 --epochs 100 --data securitycamera.yaml --weights yolov5s.pt --cache

# Use the model to detect and draw a rectangle from each class

In [ ]:
import torch
import cv2 as cv
import numpy as np

# the model location is upto the location of the model in each device
model = torch.hub.load('ultralytics/yolov5', 'custom', path='D:/Work/University Work/Year 2/Term 2/Robotic LAB 3/Final/Project/Test code/Securitycammodel.pt')

# Open Webcam
cap = cv.VideoCapture(0)

class_labels = ['car', 'human', 'motorbike']

while True:
    # Read a frame from the webcam
    ret, img = cap.read()

    # let the model read what they see in the webcam frame by frame and output the result
    results = model(img)

    # Convert results.pred to a numpy array
    pred = np.array([x.tolist() for x in results.pred[0]])

    if len(pred) > 0:
        # Loop over each detection
        for detection in pred:
            x1, y1, x2, y2, conf, cls = detection

            # Convert class index to int
            cls = int(cls)

            # Get class label and confidence score from each class
            class_label = class_labels[cls]
            confidence = f'{conf:.2f}'

            # Draw a rectangle box

            if class_label == "car":
                cv.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            
            if class_label == "human":
                cv.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

            if class_label == "motorbike":
                cv.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

            # Display class label and confidence on the screen beside the rectangle
            label = f'{class_label} {confidence}'
            cv.putText(img, label, (int(x1), int(y1) - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)

    # Display the output image
    cv.imshow('Output', img)

    # Break the loop if 'q' key is pressed
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv.destroyAllWindows()
